<a href="https://colab.research.google.com/github/CSID-DGU/2020-2-OSSP1-WhatsUp-5/blob/master/model/generateNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 cd /content/drive/Shareddrives/와썹_공개SW/GPT2_MNews/

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

In [ ]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [ ]:
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'
tokenizer = SentencepieceTokenizer(TOKENIZER_PATH, num_best=0, alpha=0)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

## 텍스트 생성-코드

In [ ]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])

In [ ]:
from tqdm import tqdm
def generate_sent(seed_word, model, max_step=400, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    for _ in tqdm(range(max_step)):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        #print("생성된 토큰 : "+gen)
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)
    return sent

## 텍스트 생성-주석



```
# generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
```
-see_word: 문장 생성의 시작 단어

-model: 문장 생성을 수행할 모델

-max_step: 생성 횟수를 제한

-greedy: 모델 출력 결과에 대해 유연하게 문장 생성을 해줄 수 있는지 선택할 수 있도록

*   greedy=true: 문장 출력 결과에 대해 가장 확률이 높은 단어만 선택
*   greedy=false: 출력한 단어 가운데 확률 또는 순위가 높은 단어만 선택해 무작위 생성

+) top_k와 top_p 파라미터:  false인 경우 사용. top_k는 확률이 높은 순서대로 k번째까지 필터링. top_p는 일정 확률값 이상인 단어에 대해 필터링



```
#  sent = seed_word
   toked = tokenizer(sent)
```

문장 시작 단어를 변수에 할당하고 토크나이즈


```
# for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
```
문장 생성을 할 수 있는 반복문. 토크나이즈된 단어를 인덱스로 변환하고 모델에 입력값으로 넣어 출력값을 받음. 모델의 출력값에 대해서는 문장에서 마지막 단어만 선택

```
# if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)
```
생성된 텍스트 토큰이 문장의 끝을 알리는 </s</s>>토큰이면 생성 stop하고 앞서 만들어진 텍스트에 덧붙임






## 로스 0.35 제목 생성

In [ ]:
BASE_MODEL_PATH = './mnews_title/final_loss0.6_acc0.5'
gpt_model = GPT2Model(BASE_MODEL_PATH)

In [ ]:
generate_sent('코로나19', gpt_model,max_step=20,top_k=0, top_p=0.90)

In [ ]:
generate_sent('뇌졸중', gpt_model,max_step=20,top_k=40, top_p=0.90)

In [ ]:
generate_sent('치매', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('치매', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('우울증', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('우울증', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('고혈압', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('고혈압', gpt_model,top_k=40, top_p=0.95)

## 로스 1208 제목 생성

In [ ]:
BASE_MODEL_PATH = './mnews_title/maxlen20_loss0.36_acc0.56'
gpt_model = GPT2Model(BASE_MODEL_PATH)

In [ ]:
generate_sent('코로나19', gpt_model,top_k=0, top_p=0.90)

In [ ]:
generate_sent('코로나19', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('코로나19', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('뇌졸중', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('뇌졸중', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('치매', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('치매', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('우울증', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('우울증', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('고혈압', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('고혈압', gpt_model,top_k=40, top_p=0.95)

## 로스 1207 제목 생성

In [ ]:
BASE_MODEL_PATH = './mnews_title/test_maxlen10_loss1.0_acc3.4'
gpt_model = GPT2Model(BASE_MODEL_PATH)

In [ ]:
generate_sent('코로나19', gpt_model,top_k=0, top_p=0.90)

In [ ]:
generate_sent('코로나19', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('코로나19', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('뇌졸중', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('뇌졸중', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('치매', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('치매', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('우울증', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('우울증', gpt_model,top_k=40, top_p=0.95)

In [ ]:
generate_sent('고혈압', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('고혈압', gpt_model,top_k=0, top_p=0.75)

## 본문생성(1)

In [ ]:
BASE_MODEL_PATH = './350_model_path/2.31_0.389_tf2_gpt2_350_finetuned_model'
gpt_model = GPT2Model(BASE_MODEL_PATH)

In [ ]:
generate_sent('고혈압', gpt_model,top_k=0, top_p=0.90)

In [ ]:
generate_sent('우울증?', gpt_model,top_k=40, top_p=0.90)

In [ ]:
generate_sent('뇌졸중', gpt_model,top_k=0, top_p=0.90)

In [ ]:
generate_sent('코로나19 환자 분류시 ‘맥박, 혈압, 호흡, 체온, 의식’ 본다', gpt_model,top_k=40, top_p=0.90)

In [ ]:
csv